<a href="https://colab.research.google.com/github/waltz2u/bd/blob/master/Feature_Extraction_and_Transformation_with_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Setting things up

In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,78

In [3]:
!ls

gdrive	sample_data  spark-2.4.5-bin-hadoop2.7	spark-2.4.5-bin-hadoop2.7.tgz


In [4]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [5]:
!java -version 

openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Feature Extraction and Transformation with MLlib
Read for more about feature Extraction and Transformation with MLlib
https://spark.apache.org/docs/latest/mllib-feature-extraction.html

In [0]:
import findspark
findspark.init("/content/spark-2.4.5-bin-hadoop2.7")
from pyspark.sql import SparkSession
# Sets the Spark master URL to connect to, such as "local" to run locally, 
# "local[4]" to run locally with 4 cores, or "spark://master:7077" to run on a Spark standalone cluster.
spark = SparkSession.builder.master("local[*]").getOrCreate() #

## TF-IDF

In [8]:
data = spark.createDataFrame([
    (0.0, "We start learning Spark"),
    (0.0, "Python and R have API to work with it"),
    (1.0, "We use Kmeans to cluster stock movement"),
    (1.0, "We use logistic regression to predict bankruptcy")
], ["label", "content"])

data.select('content').show()

+--------------------+
|             content|
+--------------------+
|We start learning...|
|Python and R have...|
|We use Kmeans to ...|
|We use logistic r...|
+--------------------+



In [9]:
data.select('content').rdd.collect()

[Row(content='We start learning Spark'),
 Row(content='Python and R have API to work with it'),
 Row(content='We use Kmeans to cluster stock movement'),
 Row(content='We use logistic regression to predict bankruptcy')]

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer,StopWordsRemover
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes   
from pyspark.sql import SQLContext, Row
from pyspark.mllib.evaluation import MulticlassMetrics
from operator import itemgetter

# tokenizer to create a "terms" column so for example:
# from content=u'We start learning Spark'  we have terms=[u'we', u'start', u'learning', u'spark']
tokenizer = Tokenizer(inputCol="content", outputCol="terms")
termsData = tokenizer.transform(data)

# remover to remove stop words that don't contribute so for example
# from terms=[u'we', u'start', u'learning', u'spark'] we have filtered=[u'start', u'learning', u'spark']
remover = StopWordsRemover(inputCol="terms", outputCol="filtered")
filteredTermsData = remover.transform(termsData)

# http://spark.apache.org/docs/latest/ml-features.html
# Both HashingTF and CountVectorizer can be used to generate the term frequency vectors.
# HashingTF is a Transformer which takes sets of terms and converts those sets into fixed-length feature vectors. In text processing, a “set of terms” might # be a bag of words. HashingTF utilizes the hashing trick. 
# so from filtered=[u'start', u'learning', u'spark'] we have rawFeatures=SparseVector(262144, {29470: 1.0, 62173: 1.0, 181346: 1.0})
tf = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10).transform(filteredTermsData)

# IDF: IDF is an Estimator which is fit on a dataset and produces an IDFModel. The IDFModel takes feature vectors (generally created from HashingTF or 
# CountVectorizer) and scales each column. Intuitively, it down-weights columns which appear frequently in a corpus.
idf = IDF(inputCol="rawFeatures", outputCol="features").fit(tf)

# TF-IDF
tfidf = idf.transform(tf)

labels = data.rdd.map(
    lambda doc: doc["label"]  # Standard Python dict access 
)

Now take a look at tokenized data in termsData. 

tokenizer to create a "terms" column so for exampl, from content=u'We start learning Spark', we have terms=[u'we', u'start', u'learning', u'spark']

In [11]:
termsData.collect()

[Row(label=0.0, content='We start learning Spark', terms=['we', 'start', 'learning', 'spark']),
 Row(label=0.0, content='Python and R have API to work with it', terms=['python', 'and', 'r', 'have', 'api', 'to', 'work', 'with', 'it']),
 Row(label=1.0, content='We use Kmeans to cluster stock movement', terms=['we', 'use', 'kmeans', 'to', 'cluster', 'stock', 'movement']),
 Row(label=1.0, content='We use logistic regression to predict bankruptcy', terms=['we', 'use', 'logistic', 'regression', 'to', 'predict', 'bankruptcy'])]

The remover is to remove stop words that don't contribute so for example, from terms=[u'we', u'start', u'learning', u'spark'] we have filtered=[u'start', u'learning', u'spark']

In [12]:
filteredTermsData.collect()

[Row(label=0.0, content='We start learning Spark', terms=['we', 'start', 'learning', 'spark'], filtered=['start', 'learning', 'spark']),
 Row(label=0.0, content='Python and R have API to work with it', terms=['python', 'and', 'r', 'have', 'api', 'to', 'work', 'with', 'it'], filtered=['python', 'r', 'api', 'work']),
 Row(label=1.0, content='We use Kmeans to cluster stock movement', terms=['we', 'use', 'kmeans', 'to', 'cluster', 'stock', 'movement'], filtered=['use', 'kmeans', 'cluster', 'stock', 'movement']),
 Row(label=1.0, content='We use logistic regression to predict bankruptcy', terms=['we', 'use', 'logistic', 'regression', 'to', 'predict', 'bankruptcy'], filtered=['use', 'logistic', 'regression', 'predict', 'bankruptcy'])]

Both HashingTF and CountVectorizer can be used to generate the term frequency vectors.

HashingTF is a Transformer which takes sets of terms and converts those sets into fixed-length feature vectors. In text processing, a “set of terms” might be a bag of words. HashingTF utilizes the hashing trick. So from filtered=[u'start', u'learning', u'spark'] we have rawFeatures=SparseVector(10, {2: 1.0, 5: 1.0, 6: 1.0}))

Note that the numFeatures=10 in HashingTF decides the size of SparseVector

In [13]:
tf.collect()

[Row(label=0.0, content='We start learning Spark', terms=['we', 'start', 'learning', 'spark'], filtered=['start', 'learning', 'spark'], rawFeatures=SparseVector(10, {2: 1.0, 5: 1.0, 6: 1.0})),
 Row(label=0.0, content='Python and R have API to work with it', terms=['python', 'and', 'r', 'have', 'api', 'to', 'work', 'with', 'it'], filtered=['python', 'r', 'api', 'work'], rawFeatures=SparseVector(10, {0: 1.0, 4: 1.0, 7: 1.0, 9: 1.0})),
 Row(label=1.0, content='We use Kmeans to cluster stock movement', terms=['we', 'use', 'kmeans', 'to', 'cluster', 'stock', 'movement'], filtered=['use', 'kmeans', 'cluster', 'stock', 'movement'], rawFeatures=SparseVector(10, {0: 1.0, 2: 1.0, 4: 1.0, 5: 1.0, 9: 1.0})),
 Row(label=1.0, content='We use logistic regression to predict bankruptcy', terms=['we', 'use', 'logistic', 'regression', 'to', 'predict', 'bankruptcy'], filtered=['use', 'logistic', 'regression', 'predict', 'bankruptcy'], rawFeatures=SparseVector(10, {3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 9: 1.0}))

HashingTF takes sets of terms and converts those sets into fixed-length feature vectors. In text processing, a “set of terms” might be a bag of words. HashingTF utilizes the hashing trick. A raw feature is mapped into an index (term) by applying a hash function. The hash function used here is MurmurHash 3

Now look at tfidf, in features, this is where data is ready for modeling

In [14]:
tfidf.collect()

[Row(label=0.0, content='We start learning Spark', terms=['we', 'start', 'learning', 'spark'], filtered=['start', 'learning', 'spark'], rawFeatures=SparseVector(10, {2: 1.0, 5: 1.0, 6: 1.0}), features=SparseVector(10, {2: 0.5108, 5: 0.2231, 6: 0.5108})),
 Row(label=0.0, content='Python and R have API to work with it', terms=['python', 'and', 'r', 'have', 'api', 'to', 'work', 'with', 'it'], filtered=['python', 'r', 'api', 'work'], rawFeatures=SparseVector(10, {0: 1.0, 4: 1.0, 7: 1.0, 9: 1.0}), features=SparseVector(10, {0: 0.5108, 4: 0.2231, 7: 0.9163, 9: 0.2231})),
 Row(label=1.0, content='We use Kmeans to cluster stock movement', terms=['we', 'use', 'kmeans', 'to', 'cluster', 'stock', 'movement'], filtered=['use', 'kmeans', 'cluster', 'stock', 'movement'], rawFeatures=SparseVector(10, {0: 1.0, 2: 1.0, 4: 1.0, 5: 1.0, 9: 1.0}), features=SparseVector(10, {0: 0.5108, 2: 0.5108, 4: 0.2231, 5: 0.2231, 9: 0.2231})),
 Row(label=1.0, content='We use logistic regression to predict bankruptcy',

Note that if above we do not specify inputCol and outputCol, the structures would be simple, like (for testing, try it on your own)

In [15]:
tf.collect()

[Row(label=0.0, content='We start learning Spark', terms=['we', 'start', 'learning', 'spark'], filtered=['start', 'learning', 'spark'], rawFeatures=SparseVector(10, {2: 1.0, 5: 1.0, 6: 1.0})),
 Row(label=0.0, content='Python and R have API to work with it', terms=['python', 'and', 'r', 'have', 'api', 'to', 'work', 'with', 'it'], filtered=['python', 'r', 'api', 'work'], rawFeatures=SparseVector(10, {0: 1.0, 4: 1.0, 7: 1.0, 9: 1.0})),
 Row(label=1.0, content='We use Kmeans to cluster stock movement', terms=['we', 'use', 'kmeans', 'to', 'cluster', 'stock', 'movement'], filtered=['use', 'kmeans', 'cluster', 'stock', 'movement'], rawFeatures=SparseVector(10, {0: 1.0, 2: 1.0, 4: 1.0, 5: 1.0, 9: 1.0})),
 Row(label=1.0, content='We use logistic regression to predict bankruptcy', terms=['we', 'use', 'logistic', 'regression', 'to', 'predict', 'bankruptcy'], filtered=['use', 'logistic', 'regression', 'predict', 'bankruptcy'], rawFeatures=SparseVector(10, {3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 9: 1.0}))

In [16]:
tfidf.collect()

[Row(label=0.0, content='We start learning Spark', terms=['we', 'start', 'learning', 'spark'], filtered=['start', 'learning', 'spark'], rawFeatures=SparseVector(10, {2: 1.0, 5: 1.0, 6: 1.0}), features=SparseVector(10, {2: 0.5108, 5: 0.2231, 6: 0.5108})),
 Row(label=0.0, content='Python and R have API to work with it', terms=['python', 'and', 'r', 'have', 'api', 'to', 'work', 'with', 'it'], filtered=['python', 'r', 'api', 'work'], rawFeatures=SparseVector(10, {0: 1.0, 4: 1.0, 7: 1.0, 9: 1.0}), features=SparseVector(10, {0: 0.5108, 4: 0.2231, 7: 0.9163, 9: 0.2231})),
 Row(label=1.0, content='We use Kmeans to cluster stock movement', terms=['we', 'use', 'kmeans', 'to', 'cluster', 'stock', 'movement'], filtered=['use', 'kmeans', 'cluster', 'stock', 'movement'], rawFeatures=SparseVector(10, {0: 1.0, 2: 1.0, 4: 1.0, 5: 1.0, 9: 1.0}), features=SparseVector(10, {0: 0.5108, 2: 0.5108, 4: 0.2231, 5: 0.2231, 9: 0.2231})),
 Row(label=1.0, content='We use logistic regression to predict bankruptcy',

A little bit more about SparseVector: 

In the following example for SparseVector, the second argument of Vectors.sparse is an array of the indices, and the third argument is the array of the actual values in those indices.

In [17]:
from pyspark.mllib.linalg import SparseVector
vec = SparseVector(3, [0, 2], [1.0, 3.0])
vec

SparseVector(3, {0: 1.0, 2: 3.0})

another way to create a SparseVector

In [18]:
from pyspark.mllib.linalg import Vectors
vec = Vectors.sparse(3, [0, 2], [1.0, 3.0])
vec

SparseVector(3, {0: 1.0, 2: 3.0})

If you would like to see what a SparseVector is in the form of a DenseVector

In [19]:
from pyspark.mllib.linalg import SparseVector, DenseVector

DenseVector(vec)

DenseVector([1.0, 0.0, 3.0])

A sparse vector is represented by two parallel arrays: indices and values. Zero entries are not stored. A dense vector is backed by a double array representing its entries

# StandardScaler

Standardizes features by scaling to unit variance and/or removing the mean using column summary statistics on the samples in the training set. This is a very common pre-processing step. 

In [0]:
from pyspark.mllib.feature import StandardScaler, StandardScalerModel
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.util import MLUtils

sc = SparkContext.getOrCreate()

data = MLUtils.loadLibSVMFile(sc, r"/content/spark-2.4.5-bin-hadoop2.7/data/mllib/sample_libsvm_data.txt")
label = data.map(lambda x: x.label)
features = data.map(lambda x: x.features)

scaler1 = StandardScaler().fit(features)
scaler2 = StandardScaler(withMean=True, withStd=True).fit(features)

# data1 will be unit variance.
data1 = label.zip(scaler1.transform(features))

# data2 will be unit variance and zero mean.
data2 = label.zip(scaler2.transform(features.map(lambda x: Vectors.dense(x.toArray()))))

Now take a look at the scaled data using take method of RDDs. Note that data1 is an RDD of SparseVector, while data2 is an RDD of DenVector using map

In [27]:
features.take(1)

[SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0, 356: 253.0, 357: 252.0

In [28]:
data1.take(1)

[(0.0,
  SparseVector(692, {127: 0.5468, 128: 1.5923, 129: 2.4354, 130: 1.7081, 131: 0.7335, 154: 0.4346, 155: 2.0985, 156: 2.2563, 157: 2.2368, 158: 2.2269, 159: 2.2555, 181: 0.4713, 182: 2.0575, 183: 2.3318, 184: 2.3761, 185: 2.1237, 186: 2.0452, 187: 2.2657, 188: 0.6339, 189: 0.1022, 207: 0.1056, 208: 0.5395, 209: 1.9268, 210: 2.2383, 211: 2.3018, 212: 2.3568, 213: 1.8002, 214: 0.7116, 215: 2.2256, 216: 2.4032, 217: 1.5931, 235: 1.5394, 236: 2.188, 237: 2.1493, 238: 2.2924, 239: 2.3889, 240: 2.3155, 241: 2.2653, 242: 0.8445, 243: 1.7094, 244: 2.2496, 245: 1.8613, 262: 0.5062, 263: 2.0796, 264: 2.2201, 265: 2.199, 266: 1.7299, 267: 1.083, 268: 2.1786, 269: 2.0345, 270: 0.4392, 271: 0.7218, 272: 2.2177, 273: 1.6764, 289: 0.4794, 290: 2.214, 291: 2.3569, 292: 2.2283, 293: 1.6322, 294: 0.1087, 295: 0.6833, 296: 1.0411, 297: 0.1941, 300: 2.277, 301: 2.3083, 302: 0.5395, 316: 0.3967, 317: 1.6059, 318: 2.3539, 319: 2.1535, 320: 1.9834, 321: 0.8017, 328: 2.4941, 329: 2.3661, 330: 1.7473, 34

In [29]:
data2.take(1)

[(0.0,
  DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1, -0.1, -0.1357, -0.1287, -0.1584, -0.1689, -0.1934, -0.1068, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1, -0.1, -0.1338, -0.136, -0.2777, -0.4448, -0.5367, -0.6242, -0.1413, 0.8975, 1.6834, 1.0786, 0.2778, -0.2784, -0.1424, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1, -0.1, -0.1238, -0.1741, -0.2875, -0.436, -0.6436, -0.804, -0.5563, 0.9701, 1.0756, 1.0633, 1.2936, 1.5017, -0.4409, -0.2643, -0.1677, -0.1, -0.1, 0.0, 0.0, 0.0, 0.0, 0

# MinMaxScaler

MinMaxScaler transforms a dataset of Vector rows, rescaling each feature to a specific range (often [0, 1]). It takes parameters:

min: 0.0 by default. Lower bound after transformation, shared by all features.
    
max: 1.0 by default. Upper bound after transformation, shared by all features.

In [30]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors

dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, -1.0]),),
    (1, Vectors.dense([2.0, 1.1, 1.0]),),
    (2, Vectors.dense([3.0, 10.1, 3.0]),)
], ["id", "features"])

scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(dataFrame)

# rescale each feature to range [min, max].
scaledData = scalerModel.transform(dataFrame)
print("Features scaled to range: [%f, %f]" % (scaler.getMin(), scaler.getMax()))
scaledData.select("features", "scaledFeatures").show()

Features scaled to range: [0.000000, 1.000000]
+--------------+--------------+
|      features|scaledFeatures|
+--------------+--------------+
|[1.0,0.1,-1.0]| [0.0,0.0,0.0]|
| [2.0,1.1,1.0]| [0.5,0.1,0.5]|
|[3.0,10.1,3.0]| [1.0,1.0,1.0]|
+--------------+--------------+

